<a href="https://colab.research.google.com/github/gogimandu1019/bigdata2023/blob/main/01_NLP_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

자연어처리 모델을 활용한 다양한 예제

- 원작자 @kiyoungkim1 [**(링크)**](https://github.com/kiyoungkim1/LMkor)

## 설치

In [1]:
!git clone https://github.com/kiyoungkim1/LMkor
!pip3 install -q transformers

Cloning into 'LMkor'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 101 (delta 17), reused 11 (delta 11), pack-reused 80
Receiving objects: 100% (101/101), 45.28 KiB | 22.64 MiB/s, done.
Resolving deltas: 100% (46/46), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00


## 필요한 모델 다운로드

In [2]:
from LMkor.examples.gpt3_generation import Inference
from LMkor.examples.bertshared_summarization import Summarize
from LMkor.examples.mask_prediction import predict

inference = Inference('kykim/gpt3-kor-small_based_on_gpt2')
summarize = Summarize('kykim/bertshared-kor-base')

## 문서 요약(Text Summarization)

In [3]:
text = '''
LG전자가 스마트폰을 담당하는 MC(모바일커뮤니케이션)사업부 분할 및 매각을 위한 법률 자문 업무를 김앤장법률사무소에 맡겼다. MC사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.
22일 인수합병(M&A)업계에 따르면 LG전자는 최근 MC사업부 분할 후 매각 방안 등을 포괄적으로 검토하기 위해 김앤장을 법률자문사로 선임한 것으로 알려졌다. 회계·실사 자문은 EY한영회계법인에 맡길 가능성이 큰 것으로 전해졌다. 김앤장 등 자문사들은 사업본부를 분할한 뒤 사업양수도나 분할사업부의 지분 매각, 지식재산권(IP) 매각 등을 놓고 검토에 들어간 것으로 알려졌다.
업계에서는 LG전자가 MC사업본부를 통매각하기보다는 ‘쪼개기 매각’에 나설 것으로 보고 있다. 스마트폰 선행기술 연구개발(R&D) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측하고 있다. 앞서 권봉석 LG전자 사장은 사내 메시지를 통해 임직원에게 “현재 모든 가능성을 열어 두고 사업 운영방향을 면밀히 검토하고 있다”고 밝히며 매각 추진을 암시했다. M&A업계 관계자는 “거래가 성사되기도 전에 사업 전면 재검토를 공식화한 것은 상당히 이례적”이라며 “향후 매각이 잘 이뤄지지 않더라도 모바일 사업을 철수하겠다는 배수진을 둔 것으로 보인다”고 설명했다.
다만 원매자를 찾기가 쉽지 않을 것이란 전망이 우세하다. LG전자 모바일 사업은 한때 글로벌시장에서 톱5 안에 드는 기술력을 인정받았지만 누적 적자만 5조원에 달하고 있다. 업계에서 평가하는 MC사업부의 가치도 5000억원대에서 수조원대까지 편차가 상당히 크다.
상대적으로 해외 원매자들의 인수의사가 더 확실한 것으로 알려지고 있다. 북미사업 등 글로벌 시장 확장을 원하는 후발기업들이 주요 대상이다. 베트남의 빈그룹과 중국 기업 등이 유력하게 거론된다. 증권업계를 중심으로는 스마트 기기를 연결하는 사물인터넷(IoT) 사업을 염두에 둔 구글, 페이스북 같은 미국 정보기술(IT) 기업들도 원매자 후보군으로 꼽고 있다.
'''

summarize(text)

22일 인수합병 ( m & a ) 업계에 따르면 스마트폰 선행기술 연구개발 ( r & d ) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측되고 있으며 업계에서는 mc사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.


신문기사의 본문을 발췌해오는 크롤링 로직 구현

In [4]:
# 신문기사의 URL
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004886133'

In [5]:
from bs4 import BeautifulSoup
import requests
import re

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')
body = soup.find_all(id='newsct_article')

text = re.sub('[^A-Za-z0-9가-힣 ]', '', body[0].text)
text

' 삼성 인사제도 혁신 삼성전자가 이재용의 뉴 삼성을 뒷받침하는 인사 혁신을 단행했다 직급별 표준 체류기간과 승격 포인트를 폐지해 과감한 발탁 승진이 가능하도록 한 것이 특징이다 삼성전자는 29일 연공서열을 타파하 는 인사 제도 혁신안을 마련하고 내년부터 시행한다고 밝혔다 젊은 경영진을 조기에 육성하는 삼성형 패스트트랙이 시행되면 30대 임원 40대 최고경영자CEO가 탄생할 것으로 예상된다직급표기 삭제승진연한 폐지삼성 실리콘밸리식 조직 탈바꿈이재용의 뉴삼성 인사혁신직급 대신 전문성으로만 평가능력 있으면 빠르게 발탁 승진전무 없애고 부사장으로 통합제조업식 연공서열 문화 없애유연수평적인 기업문화 구축다른부서 이동 사내FA로다양한 직무경험 기회 제공  이재용 삼성전자 부회장이 뉴 삼성을 선언한 뒤 처음으로 단행된 이번 삼성전자 인사제도 개편은 연공서열을 없애고 수평적이고 유연한 기업문화를 조성하는 것이 핵심이다 삼성전자가 제조업에 총력을 기울이던 시기에 만들어진 기업문화를 세계적인 정보통신기술ICT 기업에 맞게 새롭게 개편하는 것이다 이번 인사제도 혁신안에서 눈에 띄는 것은 삼성형 패스트트랙이다 부사장전무 직급을 부사장으로 통합해 임원 직급 단계를 축소하고 직급별 표준 체류기간이나 근무 연한을 없앤 것이 핵심이다 여기에는 나이와 상관없이 젊고 유능한 인재를 발탁하고 능력 있는 경영자를 조기에 배출하겠다는 의지가 담겨 있다 지난 9월 말 기준 삼성전자 임원은 모두 1080명이다 이 가운데 부사장은 67명 전무는 129명이다 부사장전무 직급을 통합하면 삼성전자에는 200여 명에 가까운 차기 최고경영자CEO 후보군이 생기게 된다이경묵 서울대 경영대학원 교수는 전무나 부사장 등 특정 직급과 연봉에 국한하지 않고 파격적으로 외부 인사를 스카우트할 수 있는 기반을 마련한 것이라며 인재 영입 전쟁에서 유리한 고지를 선점할 수 있게 됐다고 말했다특히 직급별 근무 연한을 폐지하는 대신 승격 세션을 도입해 젊은 임원 탄생 가능성도 높였다 삼성전자 직급 제도는 CLCareer Leve

In [6]:
def summerize_news(url):
    print(f'원문 링크: {url}\n')
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    body = soup.find_all(id='newsct_article')
    if len(body) > 0:
        text = re.sub('[^A-Za-z0-9가-힣 ]', '', body[0].text)
        text = text[:512]
        print('[요약]')
        summarize(text)
    else:
        print("에러ㅜ")

In [7]:
summerize_news(url)

원문 링크: https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004886133

[요약]
삼성전자는 이재용 삼성전자 부회장이 뉴 삼성을 선언한 뒤 처음으로 연공서열을 없애고 수평적이고 유연한 기업문화를 조성하는 인사 제도 혁신안을 마련하고 내년부터 시행한다고 밝혔다.


## 마스킹 단어 예측

In [8]:
text = '이거 <mask> 재밌네?'

predict(text)

kykim/bert-kor-base: 참 / 진짜 / 되게
kykim/albert-kor-base: 진짜 / 너무 / 정말
bert_multilingual: 리 / 지 / 가
xlm: 완전 / 왜 / 진짜


In [9]:
text = '손흥민은 <mask> 의 축구선수이다'

predict(text)

kykim/bert-kor-base: 대한민국 / 우리나라 / 독일
kykim/albert-kor-base: 대한민국 / 일본 / 미국
bert_multilingual: 대한민국의 / 대한민국 / 일본
xlm: 대한민국 / 한국 / 독일


In [10]:
text = '대한민국의 수도는 <mask> 입니다'

predict(text)

kykim/bert-kor-base: 대한민국 / 서울 / 인천
kykim/albert-kor-base: 베트남 / 대한민국 / 중국
bert_multilingual: 대한민국 / 대한민국의 / 그
xlm: 서울 / 제주 / 인천


## 뒷 문장 예측

In [11]:
text = '인생이'

inference(text, howmany=3)

0: 인생이 우리 가족에게 주는 선물은 어떤 걸까..??!!? ㅎㅎ ( 주 ) 휴테크는 지난 5월 가정의 달을 맞아 휴먼케어 ( healthcare ) 서비스인 휴머노이드 로봇 휴비스 ( h
1: 인생이 얼마나 고달프고통스러운 일인지 알죠. 이 일을 겪고 나서 이 ~ 제가 이 자리에서 그 ~ 이 얘기를 한 적이 있거든요.
2: 인생이 행복으로 가득 차 있는 것 같아요.. ㅎㅎ!! ㅎㅎ.!.?. ~. ^ ^. ; ;. (


In [12]:
user_input = input('시작 단어를 입력해 주세요: ')
inference(user_input, howmany=3)

시작 단어를 입력해 주세요: 나 오늘 하루종일 코딩하느라 힘들었다. 인생이..
0: 나 오늘 하루종일 코딩하느라 힘들었다. 인생이.. 어제에 이어 오늘 또. 어제의 일을 반복하다 오늘 다시. 또 코딩. 오늘도 역시. 오늘 코딩에 대해 얘기할꺼다. 내가 만든 코딩은. 이렇게. 내 손으로
1: 나 오늘 하루종일 코딩하느라 힘들었다. 인생이.. 안녕하세요!! 요즘 코로나 때문에 사회적 거리두기가 2단계로 하향되면서 집콕만 하는 날이 많아졌죠 ~ 집에서 할 수 있는 놀이가 없을까 하고 찾다가 보드게임이 떠오르더라구요그래서 집에서 놀
2: 나 오늘 하루종일 코딩하느라 힘들었다. 인생이.. 오늘은 제가 요즘 푹 빠져있는 코딩프로그램 하나를 소개해드리도록 하겠습니다! 바로 코딩을 좋아하는 분들이라면 다들 알고 계실텐데요, 저는 이번에 이 프로그램을 알게 된 이후로 코딩 프로그램만 주구장창 사용하고 있는데요
